In [1]:
import pandas as pd
import numpy as np

In [2]:
df = pd.read_csv('data/df.csv')
print(df.shape)
df.head()

(236867, 16)


c:\users\joey\learn-env\lib\site-packages\IPython\core\interactiveshell.py:3049: DtypeWarning: Columns (3,4) have mixed types. Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


,Unnamed: 0,% to Dems,% to Repubs,0,1,Cycle,Democrats,Individuals,PACs,Republicans,Soft (Indivs),Soft (Orgs),Total,Unnamed: 0.1,org_id,org_name
0,0,0%,100%,NaN,NaN,2008.0,$0,"$10,600",$0,"$10,600",$0,$0,"$10,600",1,D000000001,Joseph E Seagram & Sons
1,1,0%,100%,NaN,NaN,2006.0,$0,$250,$0,$250,$0,$0,$250,2,D000000001,Joseph E Seagram & Sons
2,2,80%,20%,NaN,NaN,2004.0,"$2,000","$2,500",$0,$500,$0,$0,"$2,500",3,D000000001,Joseph E Seagram & Sons
3,3,61%,39%,NaN,NaN,1998.0,"$1,005,124","$478,511","$411,290","$643,923","$187,358","$576,488","$1,653,647",4,D000000001,Joseph E Seagram & Sons
4,4,67%,33%,NaN,NaN,1996.0,"$1,766,279","$368,191","$308,875","$861,370","$929,899","$1,020,684","$2,627,649",5,D000000001,Joseph E Seagram & Sons


In [3]:
fedcon_grouped = pd.read_csv('data/fedcon_grouped.csv')
print(fedcon_grouped.shape)
fedcon_grouped.head()

(328186, 12)


,Unnamed: 0,recipient_parent_name1,year,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size
0,0,03SLINDUSTRIESINC,2017,99705.97,99705.97,2.0,0.0,1.0,1,03SL INDUSTRIES INC,NaN,SMALL BUSINESS
1,1,03SLINDUSTRIESINC,2018,398823.88,398823.88,8.0,0.0,1.0,1,03SL INDUSTRIES INC,NaN,SMALL BUSINESS
2,2,0BASEDESIGNLLC,2017,99964.47,149712.55,999.0,2.0,50000.0,1,0 BASE DESIGN LLC,NaN,SMALL BUSINESS
3,3,10001100WILSONOWNERLLC,2018,38412.00,38412.00,3.0,1.0,22500000.0,3,1000-1100 WILSON OWNER LLC,NaN,OTHER THAN SMALL BUSINESS
4,4,10001NEWHAMPSHIREAVENUEINC,2018,128522.88,220759.92,1.0,10.0,4611000.0,1,10001 NEW HAMPSHIRE AVENUE INC,NaN,SMALL BUSINESS


In [47]:
f = fedcon_grouped.copy()
f.groupby('year').size()

year
2008     10068
2009      9993
2010     11318
2011     12546
2012     14427
2013     16167
2014     19533
2015     21293
2016     25825
2017     79822
2018    107194
dtype: int64

In [48]:
def func(x):
     return pd.Series(dict(current_award = x['current_award'].sum(),
                           potential_award = x['potential_award'].sum(), 
                           offers_received = x['offers_received'].sum(), 
                           num_employees = x['num_employees'].mean(), 
                           annual_revenue = x['annual_revenue'].mean(),
                           award_count = x['award_count'].count(),
                           recipient_name = x['recipient_name'].unique()[0], 
                           organizational_type = x['organizational_type'].unique()[0],
                           business_size = x['business_size'].unique()[0])
                           
                     )

# fc_g = fc_all.groupby(['recipient_parent_name1', 'year']).apply(f)

### Code to aggregate over election cycle years (for fed contracts data)

Runtime: ~10min

Later: rerun with 2008 as well?

In [68]:
years_list = [2009, 2011, 2013, 2015, 2017, 2019]
fedcon_allcycles = pd.DataFrame()
f = fedcon_grouped.copy()#[:100]

for y in years_list:
    t = f[(f.year==y) | (f.year==y+1)].groupby('recipient_parent_name1').apply(func)
    t['year'] = y+1
    fedcon_allcycles = pd.concat([fedcon_allcycles, t], sort=False)
    
fedcon_allcycles = fedcon_allcycles.reset_index()

In [69]:
print(fedcon_allcycles.shape)
fedcon_allcycles.head()

(220207, 11)


,recipient_parent_name1,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size,year
0,12THKASSOCIATESLIMITEDPARTNERHIP,224760.00,0.00,0.0,85.000000,1.409277e+07,2.0,12TH & K ASSOCIATES LIMITED PARTNERHIP,OTHER,SMALL BUSINESS,2010
1,1615LLLLLC,291158.00,0.00,0.0,70.000000,6.500000e+06,2.0,1615-LLL LLC,NaN,SMALL BUSINESS,2010
2,16THKHOTELSUBLESSEELP,73006.72,0.00,6.0,0.000000,0.000000e+00,2.0,16TH & K HOTEL SUBLESSEE LP,PARTNERSHIP,SMALL BUSINESS,2010
3,17MACHINERYLLC,272327.75,272327.75,29.0,3.666667,4.860830e+05,2.0,17 MACHINERY LLC,NaN,SMALL BUSINESS,2010
4,17THSTPHOTOSUPPLYINC,289655.59,289655.59,14046.0,72.444444,3.285714e+07,2.0,"17TH ST. PHOTO SUPPLY, INC.",NaN,SMALL BUSINESS,2010


In [99]:
# fedcon_allcycles.to_csv('data/fedcon_allcycles.csv', index=False)

In [101]:
fedcon_allcycles.head()

,recipient_parent_name1,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size,year
0,12THKASSOCIATESLIMITEDPARTNERHIP,224760.00,0.00,0.0,85.000000,1.409277e+07,2.0,12TH & K ASSOCIATES LIMITED PARTNERHIP,OTHER,SMALL BUSINESS,2010
1,1615LLLLLC,291158.00,0.00,0.0,70.000000,6.500000e+06,2.0,1615-LLL LLC,NaN,SMALL BUSINESS,2010
2,16THKHOTELSUBLESSEELP,73006.72,0.00,6.0,0.000000,0.000000e+00,2.0,16TH & K HOTEL SUBLESSEE LP,PARTNERSHIP,SMALL BUSINESS,2010
3,17MACHINERYLLC,272327.75,272327.75,29.0,3.666667,4.860830e+05,2.0,17 MACHINERY LLC,NaN,SMALL BUSINESS,2010
4,17THSTPHOTOSUPPLYINC,289655.59,289655.59,14046.0,72.444444,3.285714e+07,2.0,"17TH ST. PHOTO SUPPLY, INC.",NaN,SMALL BUSINESS,2010


In [103]:
def func(x):
     return pd.Series(dict(current_award = x['current_award'].sum(),
                           potential_award = x['potential_award'].sum(), 
                           offers_received = x['offers_received'].sum(), 
                           num_employees = x['num_employees'].mean(), 
                           annual_revenue = x['annual_revenue'].mean(),
                           award_count = x['award_count'].count(),
                           recipient_name = x['recipient_name'].unique()[0], 
                           organizational_type = x['organizational_type'].unique()[0],
                           business_size = x['business_size'].unique()[0])
                           
                     )

fc_ga = fedcon_allcycles.groupby(['recipient_parent_name1']).apply(func)

In [140]:
fedcon_100mil = fc_ga[fc_ga.current_award > 100000000]
fedcon_100mil = fedcon_100mil.reset_index()
fedcon_100mil.head()

,recipient_parent_name1,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size
0,10TANKERAIRCARRIERLLC,4.230000e+09,4.605000e+09,421.0,12.769147,8.660938e+06,3,10 TANKER AIR CARRIER LLC,OTHER,SMALL BUSINESS
1,1901GROUPLLC,5.689863e+08,1.734696e+09,13133.0,25.877293,2.136121e+06,2,1901 GROUP LLC,OTHER,SMALL BUSINESS
2,1SOURCECONSULTINGINC,8.811237e+08,5.828028e+11,21714.0,463.390464,1.802715e+08,3,1 SOURCE CONSULTING INC.,NaN,SMALL BUSINESS
3,1STAMERICANSYSTEMSANDSERVICESLLC,1.346269e+08,3.918236e+12,4035.0,51.624953,3.001582e+06,3,1ST AMERICAN SYSTEMS AND SERVICES LLC,NaN,SMALL BUSINESS
4,2020COMPANYLLC,1.839248e+08,2.523483e+09,16450.0,107.362807,7.203952e+06,2,2020 COMPANY LLC,NaN,OTHER THAN SMALL BUSINESS


In [141]:
# conlist = fedcon_10mil.recipient_name.unique()
conlist = fedcon_100mil.recipient_parent_name1.unique()
len(conlist)

5168

#### Filter fedcon by awards greater than 10mil

In [142]:
fedcon_final = fedcon_allcycles[fedcon_allcycles.recipient_parent_name1.isin(conlist)]
fedcon_final.to_csv('data/fedcon_parent_100m_final.csv', index=False)

In [4]:
pd.read_csv('data/fedcon_parent_100m_final.csv').shape

(16841, 11)

### Merge fedcon with open secrets summaries

In [5]:
opensecrets = df.copy()
opensecrets.head()

,Unnamed: 0,% to Dems,% to Repubs,0,1,Cycle,Democrats,Individuals,PACs,Republicans,Soft (Indivs),Soft (Orgs),Total,Unnamed: 0.1,org_id,org_name
0,0,0%,100%,NaN,NaN,2008.0,$0,"$10,600",$0,"$10,600",$0,$0,"$10,600",1,D000000001,Joseph E Seagram & Sons
1,1,0%,100%,NaN,NaN,2006.0,$0,$250,$0,$250,$0,$0,$250,2,D000000001,Joseph E Seagram & Sons
2,2,80%,20%,NaN,NaN,2004.0,"$2,000","$2,500",$0,$500,$0,$0,"$2,500",3,D000000001,Joseph E Seagram & Sons
3,3,61%,39%,NaN,NaN,1998.0,"$1,005,124","$478,511","$411,290","$643,923","$187,358","$576,488","$1,653,647",4,D000000001,Joseph E Seagram & Sons
4,4,67%,33%,NaN,NaN,1996.0,"$1,766,279","$368,191","$308,875","$861,370","$929,899","$1,020,684","$2,627,649",5,D000000001,Joseph E Seagram & Sons


#### Unique set of orgs with PACs

In [6]:
orgs_w_pacs = set(opensecrets[opensecrets['PACs'] != '$0']['org_name'].dropna().values)
len(orgs_w_pacs)

5923

#### Subset contributions by rows with PAC money

In [7]:
opensecrets_pacs = opensecrets[opensecrets.PACs != '$0']
print(opensecrets_pacs.shape)
opensecrets_pacs.head()

(41296, 16)


,Unnamed: 0,% to Dems,% to Repubs,0,1,Cycle,Democrats,Individuals,PACs,Republicans,Soft (Indivs),Soft (Orgs),Total,Unnamed: 0.1,org_id,org_name
3,3,61%,39%,NaN,NaN,1998.0,"$1,005,124","$478,511","$411,290","$643,923","$187,358","$576,488","$1,653,647",4,D000000001,Joseph E Seagram & Sons
4,4,67%,33%,NaN,NaN,1996.0,"$1,766,279","$368,191","$308,875","$861,370","$929,899","$1,020,684","$2,627,649",5,D000000001,Joseph E Seagram & Sons
5,5,73%,27%,NaN,NaN,1994.0,"$406,924","$103,070","$216,370","$148,070","$100,000","$135,554","$554,994",6,D000000001,Joseph E Seagram & Sons
6,6,45%,55%,NaN,NaN,1992.0,"$497,470","$118,285","$258,250","$610,702","$650,000","$81,637","$1,108,172",7,D000000001,Joseph E Seagram & Sons
7,7,72%,28%,NaN,NaN,1990.0,"$165,400","$71,850","$157,025","$63,475",NaN,NaN,"$228,875",8,D000000001,Joseph E Seagram & Sons


Text cleaning:
- lower
- punctuation
- freq words

In [26]:
' '.join(o.org_name_clean).split()

['josepheseagramsons',
 'josepheseagramsons',
 'josepheseagramsons',
 'josepheseagramsons',
 'josepheseagramsons',
 'gtecorp',
 'gtecorp',
 'gtecorp',
 'gtecorp',
 'gtecorp',
 'americangeneralcorp',
 'americangeneralcorp',
 'americangeneralcorp',
 'americangeneralcorp',
 'americangeneralcorp',
 'americangeneralcorp',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'chevron',
 'coastalcorp',
 'coastalcorp',
 'coastalcorp',
 'coastalcorp',
 'coastalcorp',
 'coastalcorp',
 'pacifictelesisgroup',
 'pacifictelesisgroup',
 'pacifictelesisgroup',
 'pacifictelesisgroup',
 'nynexcorp',
 'nynexcorp',
 'nynexcorp',
 'nynexcorp',
 'mcainc',
 'mcainc',
 'mcainc',
 'championinternationalcorp',
 'championinternationalcorp',
 'championinternationalcorp',
 'championinternationalcorp',
 'championinternationalcorp',
 'salomonbrothers',
 'salomonbrothers',
 'salomonbrothers'

In [32]:
import json
with open('data/common_terms.json') as f:
    common_terms = json.load(f)

JSONDecodeError: Expecting property name enclosed in double quotes: line 1 column 2 (char 1)

In [39]:
from common_terms import *
list(common_terms.keys())

['INC.',
 'LLC',
 'INC',
 'CORPORATION',
 'SERVICES',
 '&',
 'CONSTRUCTION',
 'GROUP',
 'SOLUTIONS',
 'COMPANY',
 'SYSTEMS',
 'OF',
 'THE',
 'TECHNOLOGIES',
 'AND',
 'INCORPORATED',
 'TECHNOLOGY',
 'ASSOCIATES',
 'INTERNATIONAL',
 'CONSULTING',
 'MANAGEMENT',
 'JOINT',
 'VENTURE',
 'ENGINEERING',
 'JV',
 'RESEARCH',
 'L.L.C.',
 'ENTERPRISES',
 'INDUSTRIES',
 'CO.',
 'CORP.',
 'UNIVERSITY',
 'CONTRACTING',
 'A',
 'AMERICAN',
 'GLOBAL',
 'HOLDINGS',
 'SERVICE',
 'CORP',
 'HEALTH',
 'SECURITY',
 'ENVIRONMENTAL',
 'FOR',
 'INFORMATION',
 'LTD.',
 'MEDICAL',
 'CENTER',
 'ADVANCED',
 'FEDERAL',
 'GOVERNMENT',
 'STATE',
 'SERVICES,',
 'ELECTRIC',
 'DEVELOPMENT',
 'NATIONAL',
 'TECHNICAL',
 'SUPPORT',
 'DEFENSE',
 'RESOURCES',
 'MARINE',
 'INSTITUTE',
 'ENERGY',
 'CONTRACTORS',
 'DESIGN',
 'PARTNERS',
 'CO',
 'NORTH',
 '-',
 'LIMITED',
 'GROUP,',
 'SOLUTIONS,',
 'LTD',
 'PRODUCTS',
 'NEW',
 'LOGISTICS',
 'COMMUNICATIONS',
 'USA',
 'PACIFIC',
 'CONSULTANTS',
 'INTEGRATED',
 'GENERAL',
 'CITY',


In [ ]:
freq = ['co', 'company', 'corp', 'plc', '']

In [44]:
freq = pd.Series(' '.join(o.org_name_clean).split()).value_counts()[:70]
list(freq.index)

['assn',
 'of',
 'for',
 'pac',
 'american',
 'corp',
 'congress',
 'national',
 'inc',
 'cmte',
 'group',
 'america',
 'co',
 'international',
 'union',
 'energy',
 'financial',
 'friends',
 'fund',
 'council',
 'insurance',
 'llp',
 'services',
 'health',
 'democratic',
 'bank',
 'united',
 'industries',
 'us',
 'new',
 'life',
 'institute',
 'club',
 'leadership',
 'to',
 'growers',
 'holdings',
 'alliance',
 'california',
 'citizens',
 'al',
 'et',
 'society',
 'mutual',
 'a',
 'senate',
 'the',
 'service',
 'electric',
 'companies',
 'farm',
 'association',
 'care',
 'workers',
 'majority',
 'communications',
 'industry',
 'producers',
 'action',
 'bureau',
 'systems',
 'first',
 'healthcare',
 'gas',
 'medical',
 'campaign',
 'county',
 'federal',
 'contractors',
 'texas']

In [45]:
['assn',
 'of',
 'for',
 'pac',
 'corp',
 'corporation',
 'inc',
 'cmte',
 'group',
 'co',
 'international',
 'union',
 'llp',
 'limited',
 'company',
 'plc',
 'llc',
 'ltd',
 'to',
 'holdings',
 'al',
 'et',
 'a',
 'the',
 'companies',
 'association'
]

['assn',
 'of',
 'for',
 'pac',
 'corp',
 'corporation',
 'inc',
 'cmte',
 'group',
 'co',
 'international',
 'union',
 'llp',
 'limited',
 'company',
 'plc',
 'llc',
 'ltd',
 'to',
 'holdings',
 'al',
 'et',
 'a',
 'the',
 'companies',
 'association']

In [28]:
o = opensecrets_pacs.copy()
o['org_name_clean'] = o.org_name.str.lower().str.replace('[^\w\s]','')
o = o[o.org_name_clean.isnull() == False]
# o.org_name_clean = o.org_name.str.replace('[^\w\s]','')
o.groupby('org_name_clean').size()

org_name_clean
13th colony leadership cmte                9
159 group                                  3
1776 pac                                   3
1789 project pac                           2
21st century democrats                     4
21st century freedom pac                   6
21st century majority fund                11
21st century oncology                      2
21st century pac                           7
21st century systems                       3
314 action                                 2
340b health                                3
3com corp                                  2
3g capital                                 3
3m co                                     16
3rd  long                                  3
3t pac                                     5
500 club                                   1
54th assembly district democratic club     3
613pac                                     5
67th area democratic alliance              1
8020 national asian american pac        

In [149]:
ccs = set(opensecrets_pacs.org_name.dropna().values)
len(ccs)

5923

In [152]:
fcs = set(fedcon_final['recipient_name'].values)
len(fcs)

6682

## Divide fcs and ccs for Phoebe and Joey

In [176]:
ccs_phoebe = list(ccs)[:3000]
ccs_joey = list(ccs)[3000:]
# ccs_joey = list(ccs)[3000:]

fcs_phoebe = list(fcs)[:3300]
fcs_joey = list(fcs)[3300:]

In [ ]:
pd.DataFrame()

In [172]:
diff1 = set(fcs_joey) - set(ccs_joey)
len(diff1)

3380

In [173]:
diff2 = set(ccs_joey) - set(fcs_joey)
len(diff2)

2921

In [174]:
import stringdist as sd 

def leven(diff1, diff2):
    t1=[]
    t2=[]
    score=[]
    for i in diff1:
        for j in diff2:
            t1.append(i)
            t2.append(j)
            score.append(sd.levenshtein_norm(i, j))     
    ldf = pd.DataFrame({'t1':t1, 't2':t2, 'score':score})
    return ldf

In [175]:
leven_scores_joey = leven(diff1, diff2)

KeyboardInterrupt: 

Start: 12:32pm

In [134]:
# opensecrets_pacs.to_csv('data/opensecrets_pacs.csv', index=False)

In [123]:
orgs_w_pacs = set(opensecrets.org_name.dropna().values)
len(orgs_w_pacs)

32591

In [ ]:
orgs_w_pacs

In [74]:
fedcon = fedcon_allcycles.copy()
fedcon.head()

,recipient_parent_name1,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size,year
0,12THKASSOCIATESLIMITEDPARTNERHIP,224760.00,0.00,0.0,85.000000,1.409277e+07,2.0,12TH & K ASSOCIATES LIMITED PARTNERHIP,OTHER,SMALL BUSINESS,2010
1,1615LLLLLC,291158.00,0.00,0.0,70.000000,6.500000e+06,2.0,1615-LLL LLC,NaN,SMALL BUSINESS,2010
2,16THKHOTELSUBLESSEELP,73006.72,0.00,6.0,0.000000,0.000000e+00,2.0,16TH & K HOTEL SUBLESSEE LP,PARTNERSHIP,SMALL BUSINESS,2010
3,17MACHINERYLLC,272327.75,272327.75,29.0,3.666667,4.860830e+05,2.0,17 MACHINERY LLC,NaN,SMALL BUSINESS,2010
4,17THSTPHOTOSUPPLYINC,289655.59,289655.59,14046.0,72.444444,3.285714e+07,2.0,"17TH ST. PHOTO SUPPLY, INC.",NaN,SMALL BUSINESS,2010


In [80]:
# pd.merge(fedcon, opensecrets, how='inner', left_on=['recipient_name', 'year'], right_on=['org_name', 'Cycle'])#.shape

In [67]:
fedcon_allcycles

,recipient_parent_name1,year,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size
0,1099PRO,2012,5.000000e+04,5.000000e+04,1.0,12.000000,2.500000e+06,1.0,1099 PRO,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
1,1100CARNEGIELP,2012,3.575400e+04,3.575400e+04,6.0,104.000000,7.500000e+06,1.0,1100 CARNEGIE LP,NaN,SMALL BUSINESS
2,1000KLLC,2014,2.171118e+05,2.171118e+05,22.0,180.000000,2.720000e+07,1.0,1000 K L.L.C.,NaN,OTHER THAN SMALL BUSINESS
3,1031INC,2014,2.877000e+04,2.877000e+04,4.0,26.000000,3.353305e+06,1.0,10-31 INC,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
4,108EMERGENCYVEHICLESERVICELLC,2014,1.414768e+04,1.414768e+04,2.0,12.000000,9.024120e+05,1.0,10-8 EMERGENCY VEHICLE SERVICE LLC,PARTNERSHIP,SMALL BUSINESS
5,1099PRO,2014,1.778916e+04,1.778916e+04,1.0,16.000000,2.500000e+06,1.0,1099 PRO,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
6,10TANKERAIRCARRIERLLC,2014,7.000000e+08,7.000000e+08,80.0,12.000000,6.851254e+06,2.0,10 TANKER AIR CARRIER LLC,OTHER,SMALL BUSINESS
7,1120VERMONTAVENUEASSOCIATESLLP,2014,1.991907e+05,3.768582e+05,4.0,0.750000,2.290099e+07,1.0,1120 VERMONT AVENUE ASSOCIATES LLP,NaN,OTHER THAN SMALL BUSINESS
8,1000KLLC,2016,4.354840e+05,4.354840e+05,45.0,180.000000,2.720000e+07,2.0,"1000 K, L.L.C.",NaN,SMALL BUSINESS
9,101MOBILITYLLC,2016,2.802121e+04,2.802121e+04,314.0,435.892473,6.215054e+06,1.0,"101 MOBILITY, LLC",NaN,SMALL BUSINESS


In [64]:
fedcon_allcycles.groupby('year').size()

year
2012     2
2014     6
2016     9
2018    54
dtype: int64

In [65]:
fedcon_allcycles

,year,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size
recipient_parent_name1,,,,,,,,,,
1099PRO,2012,5.000000e+04,5.000000e+04,1.0,12.000000,2.500000e+06,1.0,1099 PRO,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
1100CARNEGIELP,2012,3.575400e+04,3.575400e+04,6.0,104.000000,7.500000e+06,1.0,1100 CARNEGIE LP,NaN,SMALL BUSINESS
1000KLLC,2014,2.171118e+05,2.171118e+05,22.0,180.000000,2.720000e+07,1.0,1000 K L.L.C.,NaN,OTHER THAN SMALL BUSINESS
1031INC,2014,2.877000e+04,2.877000e+04,4.0,26.000000,3.353305e+06,1.0,10-31 INC,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
108EMERGENCYVEHICLESERVICELLC,2014,1.414768e+04,1.414768e+04,2.0,12.000000,9.024120e+05,1.0,10-8 EMERGENCY VEHICLE SERVICE LLC,PARTNERSHIP,SMALL BUSINESS
1099PRO,2014,1.778916e+04,1.778916e+04,1.0,16.000000,2.500000e+06,1.0,1099 PRO,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS
10TANKERAIRCARRIERLLC,2014,7.000000e+08,7.000000e+08,80.0,12.000000,6.851254e+06,2.0,10 TANKER AIR CARRIER LLC,OTHER,SMALL BUSINESS
1120VERMONTAVENUEASSOCIATESLLP,2014,1.991907e+05,3.768582e+05,4.0,0.750000,2.290099e+07,1.0,1120 VERMONT AVENUE ASSOCIATES LLP,NaN,OTHER THAN SMALL BUSINESS
1000KLLC,2016,4.354840e+05,4.354840e+05,45.0,180.000000,2.720000e+07,2.0,"1000 K, L.L.C.",NaN,SMALL BUSINESS


In [52]:


t = f[(f.year==2018) | (f.year==2017)][:100].groupby('recipient_parent_name1').apply(func)
t['year'] = 2018
t

,current_award,potential_award,offers_received,num_employees,annual_revenue,award_count,recipient_name,organizational_type,business_size,year
recipient_parent_name1,,,,,,,,,,
03SLINDUSTRIESINC,498529.85,498529.85,10.0,0.000000,1.000000e+00,2,03SL INDUSTRIES INC,NaN,SMALL BUSINESS,2018
0BASEDESIGNLLC,99964.47,149712.55,999.0,2.000000,5.000000e+04,1,0 BASE DESIGN LLC,NaN,SMALL BUSINESS,2018
10001100WILSONOWNERLLC,38412.00,38412.00,3.0,1.000000,2.250000e+07,1,1000-1100 WILSON OWNER LLC,NaN,OTHER THAN SMALL BUSINESS,2018
10001NEWHAMPSHIREAVENUEINC,128522.88,220759.92,1.0,10.000000,4.611000e+06,1,10001 NEW HAMPSHIRE AVENUE INC,NaN,SMALL BUSINESS,2018
1000KLLC,646471.08,646471.08,31.0,178.750000,2.736250e+07,2,1000 K L.L.C.,NaN,SMALL BUSINESS,2018
1000LOUISIANALP,163983.52,227583.52,5.0,1.000000,5.171132e+07,2,1000 LOUISIANA LP,NaN,OTHER THAN SMALL BUSINESS,2018
1007EASTSTREETOPERATORLLC,471327.59,471327.59,2.0,110.000000,2.400000e+07,1,1007 EAST STREET OPERATOR LLC,OTHER,OTHER THAN SMALL BUSINESS,2018
1008WATSONLLC,264810.00,264810.00,24.0,15.000000,1.000000e+05,1,1008 WATSON LLC,PARTNERSHIP,SMALL BUSINESS,2018
100EHIGHLANDDRIVEOPERATINGCOMPANYLLC,61420.92,61420.92,11.0,80.000000,5.400000e+06,1,100 E HIGHLAND DRIVE OPERATING COMPANY LLC,CORPORATE NOT TAX EXEMPT,SMALL BUSINESS,2018
